In [1]:
import pandas as pd
import pyarrow as pa
from graph_tool.all import *
from graph_tool import all
import csv, math, os, pprint, sys
from collections import Counter
import pandas as pd
import time
import graph_tool as gt

In [2]:
import time
from pyspark import SparkConf, SparkContext
import findspark
findspark.init()
from pyspark.sql import SparkSession

conf = (SparkConf()
    .set("spark.driver.maxResultSize", "15g").set("spark.driver.memory","100g").
        set("spark.executor.pyspark.memory","100g").set("spark.executor.memory","100g")
        .set("spark.executor.instances","1"))

# Create new context
sc = SparkContext(conf=conf)

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import os 
#os.getcwd()
spark = SparkSession.builder.getOrCreate()
spark

In [3]:
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql.functions import count, desc , col, max ,struct, sum, row_number
def shapedf(spdf):
    return str(spdf.count())+" , "+str(len(spdf.columns))

In [4]:
g2=gt.Graph(directed=False)
#g2.load("/home/sebastian/Documents/Calls_2019_2.gt")
g2.load("/home/sebastian/Documents/Calls_2019_filtered_2.gt")

In [5]:
remove_parallel_edges(g2)

In [27]:
g2.vp["clust"]=gt.clustering.local_clustering(g2)

In [28]:
g=gt.Graph(directed=True)
g.load("/home/sebastian/Desktop/Notebooks/Kin_extended_2.gt")
#g.load("/home/sebastian/Desktop/Notebooks/Kin_Sinacofi.gt")
#g.add_vertex(n=36703940)

In [29]:
df=pd.DataFrame(g2.vp.Degree.a[g2.vp.source.a>0],columns=["Degree"])

In [32]:
df["gender"]=g.vp.gender.a[g2.vp.id.a[g2.vp.source.a>0]-1]
df["age"]=g.vp.age.a[g2.vp.id.a[g2.vp.source.a>0]-1]

In [33]:
df["clust"]=g2.vp.clust.a[g2.vp.source.a>0]
df["DegreeIn"]=g2.vp.DegreeIn.a[g2.vp.source.a>0]
df["DegreeOut"]=g2.vp.DegreeOut.a[g2.vp.source.a>0]
df["DegreeCallsIn"]=g2.vp.DegreeCallsIn.a[g2.vp.source.a>0]
df["DegreeCallsOut"]=g2.vp.DegreeCallsOut.a[g2.vp.source.a>0]
df["DegreeSecIn"]=g2.vp.DegreeSecIn.a[g2.vp.source.a>0]
df["DegreeSecOut"]=g2.vp.DegreeSecOut.a[g2.vp.source.a>0]
df["source2"]=g2.vp.source2.a[g2.vp.source.a>0]
#df["sex"]=g2.vp.sex.a[g2.vp.source.a>0]

#df["age"]=g.vp.age.a[g2.vp.id.a[g2.vp.source.a>0]-1]

In [34]:
df["Out_ratio"]=df["DegreeOut"]/(df["DegreeIn"]+df["DegreeOut"])
df["OutCall_ratio"]=df["DegreeCallsOut"]/(df["DegreeCallsIn"]+df["DegreeCallsOut"])
df["OutSec_ratio"]=df["DegreeSecOut"]/(df["DegreeSecIn"]+df["DegreeSecOut"])

In [ ]:
df=df[(df.age>18) & (df.age<90)]
#df2=df[df.callsOut>0]
grouped=df.groupby(["age","gender"]).mean()
df2=pd.DataFrame(grouped)
df2=df2.reset_index()

In [47]:
df2.head()

,age,gender,Degree,clust,DegreeIn,DegreeOut,DegreeCallsIn,DegreeCallsOut,DegreeSecIn,DegreeSecOut,source2,Out_ratio,OutCall_ratio,OutSec_ratio
0,19,0,92.491020,0.070444,43.953000,48.538021,435.290791,455.436760,48460.786397,52484.666030,2.999618,0.528950,0.515293,0.517897
1,19,1,87.797654,0.071039,42.323419,45.474235,455.652283,469.978215,66648.310850,55427.370758,3.000000,0.515776,0.505074,0.472068
2,20,0,99.481374,0.065767,46.983620,52.497754,455.985205,502.096169,50525.223514,59399.421400,2.998943,0.526777,0.516559,0.518009
3,20,1,93.186890,0.065132,44.964024,48.222866,490.110671,526.087195,69608.781707,62078.552134,2.999390,0.517748,0.512688,0.480978
4,21,0,108.734482,0.062113,51.650983,57.083499,499.601502,547.576320,54714.700906,64470.877181,2.998675,0.523404,0.515759,0.518635


In [ ]:
predictors = [
#            'age',
            'clust',
            'DegreeIn',
            'DegreeOut',
            'DegreeCallsIn',
            'DegreeCallsOut',
            'DegreeSecIn',
            'DegreeSecOut',
            "Out_ratio",
            "OutCall_ratio",
            "OutSec_ratio"]

target = "gender"

In [40]:
%matplotlib inline
import matplotlib.pyplot as plt

In [46]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
for value in predictors:

    output_file(value+'_by_Age_gender.html')

    p = figure(plot_width=600, plot_height=600)


    df_1=df2[df2.gender==0]

    p.vbar(x=df_1.age, width=0.5, bottom=0,alpha=0.5,
           top=df_1[value], color="firebrick",legend_label="male")

    df_1=df2[df2.gender==1]


    p.vbar(x=df_1.age, width=0.5, bottom=0,alpha=0.5,
           top=df_1[value], color="#50c878",legend_label="female")

    p.legend.orientation = "horizontal"

    p.xaxis.axis_label = 'Age'
    p.yaxis.axis_label =  value

    #p.add_layout(Title(text="Age", align="center"), "below")

    show(p)


Loading BokehJS ...

In [48]:
from utils2 import pandas_to_spark
from pyspark.sql.types import *

In [49]:
df=pandas_to_spark(df,sqlContext)
df=df.withColumn("gender", df.gender.cast(IntegerType())).withColumn("source2", df.source2.cast(IntegerType()))

In [42]:
from pyspark.sql.functions import udf

In [102]:
#df2=df.select("*").where("source2 == 1 OR source2 ==3")
df2=df.select("*").where("source2 == 1")
shapedf(df2)

'75071 , 14'

In [85]:
from pyspark.ml.feature import StringIndexer, VectorAssembler,StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from sklearn.metrics import confusion_matrix

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j_2.12-1.4.1.jar,xgboost4j-spark_2.12-1.4.1.jar pyspark-shell'
from sparkxgb import XGBoostClassifier
from pyspark.ml.feature import PolynomialExpansion

In [134]:
va = VectorAssembler(inputCols=predictors, outputCol="vectorized_features",handleInvalid="keep")

scaler = StandardScaler()\
         .setInputCol ("vectorized_features")\
         .setOutputCol ("features")

polyExpansion = PolynomialExpansion(degree=2, inputCol="features", outputCol="polyFeatures")
pipeline = Pipeline(stages=[va,scaler,polyExpansion])
#pipeline = Pipeline(stages=[va,scaler])

In [135]:
fitted_pipeline = pipeline.fit(df2)


In [136]:
df_prepared = fitted_pipeline.transform(df2)
df_prepared.cache()
df_prepared.count()

75071

In [137]:
trainDF, testDF = df_prepared.randomSplit([0.8, 0.2], seed=24)

In [159]:
xgboost = XGBoostClassifier(
    featuresCol="polyFeatures", 
    labelCol="gender",eta=0.1,
  maxDepth=8,
  missing=0.0,
  objective="binary:logistic",
  numRound=5,
  numWorkers=2
)

In [160]:
pipeline = Pipeline().setStages([xgboost])

In [161]:
model = pipeline.fit(trainDF)

In [162]:
model.transform(testDF).select(col("gender"), col("prediction")).show(5)

+------+----------+
|gender|prediction|
+------+----------+
|     0|       0.0|
|     0|       0.0|
|     1|       0.0|
|     0|       0.0|
|     1|       0.0|
+------+----------+
only showing top 5 rows



In [163]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [164]:
evaluator=BinaryClassificationEvaluator( rawPredictionCol="prediction",
  labelCol="gender")
evaluator.evaluate(model.transform(testDF).select(col("gender"), col("prediction")))


0.6293793291483801

In [167]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
param_grid = (
  ParamGridBuilder()
  .addGrid(xgboost.eta, [0.3,0.2,1e-1])
  .addGrid(xgboost.maxDepth, [8,9,10,11,12])
    .build()
)
cv = CrossValidator(
  estimator=xgboost,
  estimatorParamMaps=param_grid,
  evaluator=evaluator,#mce,
  numFolds=5
)

#    (max_depth, min_child_weight)
#    for max_depth in range(9,12)
#    for min_child_weight in range(5,8)

In [ ]:
cvModel = cv.fit(trainDF)
model=cvModel.bestModel
predictions = cvModel.bestModel.transform(testDF)
evaluatorMulti = MulticlassClassificationEvaluator(labelCol="gender", predictionCol="prediction")


In [142]:
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderPR"}))
print(evaluator.evaluate(predictions,{evaluator.metricName: "areaUnderROC"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "accuracy"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "recallByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "precisionByLabel"}))
print(evaluatorMulti.evaluate(predictions,{evaluatorMulti.metricName: "f1"}))

0.5829886067472347
0.6400046549350001
0.6532095800610113
0.7519739897816999
0.6688009914282763
0.6485262830842817


## Task 1 :
**Create Distribution** 20 5tiles
        
**Quantile, How much time to 10% most freq outcalled**

**Quantile, How much time to 10% most freq incalled**

****Quantile, How much %calls to 10% most freq incalled****
daily weekly time distribution(freq)
